In [33]:
from google.colab import (
    userdata,
    auth
)
auth.authenticate_user()

gemini_api_key = userdata.get('gemini_api_key')
project_id = userdata.get('project_id')

In [14]:
import requests
import json
from pprint import pp


In [3]:
# 使用 HTTP 請求直接呼叫 Gemini API
payload = {
    "contents": [
        {
            "parts": [
                {
                    "text": "你好，最近過得如何?"
                }
            ]
        }
    ],
    "generationConfig": {
        "temperature": 0  # 可以改改看 溫度
    }
}

headers = {
    "Content-Type": "application/json"
}

# Gemini API endpoint
url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?key={gemini_api_key}"

response = requests.post(url, headers=headers, data=json.dumps(payload))

obj = json.loads(response.text)

pp(obj)


{'candidates': [{'content': {'parts': [{'text': '你好！謝謝你的關心。\n'
                                                '\n'
                                                '作為一個AI，我沒有人類的感受或生活，所以沒有「過得如何」的問題。不過，我的系統運作一切正常，隨時準備好為你提供幫助。\n'
                                                '\n'
                                                '你最近過得如何呢？有什麼我可以為你服務的嗎？'}],
                             'role': 'model'},
                 'finishReason': 'STOP',
                 'index': 0}],
 'usageMetadata': {'promptTokenCount': 7,
                   'candidatesTokenCount': 64,
                   'totalTokenCount': 1156,
                   'promptTokensDetails': [{'modality': 'TEXT',
                                            'tokenCount': 7}],
                   'thoughtsTokenCount': 1085},
 'modelVersion': 'models/gemini-2.5-flash-preview-05-20',
 'responseId': '-TRSaKiXB5LbjMcPo9-4qAY'}


In [8]:
%pip install google-genai


In [34]:
from google import genai

# Only run this block for Gemini Developer API
# client = genai.Client(api_key=gemini_api_key)

# Only run this block for Vertex AI API
client = genai.Client(
    vertexai=True, project=project_id, location='global'
)

# 生成回應
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="你好，最近過得如何?",
)

print(response.text)


你好！謝謝您的問候。

我是一個大型語言模型，沒有個人感受或生活，所以沒有所謂的「過得如何」。但我隨時準備好為您提供協助和資訊。

您今天過得怎麼樣？有什麼可以幫您的嗎？


In [11]:
# 這是 completion 風格(蠻多教材仍這樣寫)
user_message = """請分類以下文字是 neutral, negative 或 positive
文字: 我覺得這個披薩實在太好吃啦
情緒:
"""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=user_message,
)

print(response.text)


情緒: **positive**

**解釋:**
*   "太好吃" (tài hǎochī) 是非常正面的形容詞，表示食物味道極佳。
*   "實在" (shízài) 和 "太" (tài) 都起到了加強語氣的作用，使得「好吃」的程度更高。
*   語氣助詞 "啦" (la) 則表達了說話者的愉悅和肯定。

綜合來看，這句話表達了對披薩強烈的喜愛和讚美之情。


In [12]:
# 可改成使用 system instruction 的風格: 把不變的整體指示放在 system instruction
# user prompt 放變動的用戶輸入

from google.genai import types

user_message = """
文字: 我覺得這個披薩實在太好吃啦
"""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=user_message,
    config=types.GenerateContentConfig(
        system_instruction="請分類以下文字是 neutral, negative 或 positive"
    )
)
print(response.text)


這段文字是 **positive**。

理由：「太好吃啦」明確表達了對披薩的喜愛和讚美，是一種正面的情緒和評價。


In [14]:
# 使用 Chat Session 進行連續對話
from google.genai import types

chat = client.chats.create(
    model='gemini-2.5-flash',
    config=types.GenerateContentConfig(
        system_instruction="You are a helpful assistant."
    )
)

# 第一輪問答
response1 = chat.send_message("誰贏得2013年的世界棒球經典賽冠軍?")
print("第一輪回答:", response1.text)


第一輪回答: 2013年的世界棒球經典賽冠軍是**多明尼加共和國**。

他們在決賽中以3比0擊敗了波多黎各，並在整個賽事中保持不敗，以完美的戰績奪冠。


In [15]:
# 延續同一個對話的 第二輪問答
response2 = chat.send_message("那2017年呢?")
print("第二輪回答:", response2.text)


第二輪回答: 2017年的世界棒球經典賽冠軍是**美國**。

他們在決賽中以8比0擊敗了波多黎各，這也是美國隊首次贏得世界棒球經典賽的冠軍。


In [16]:
# 延續同一個對話的 第三輪問答
response3 = chat.send_message("美國隊贏過幾次冠軍?")
print("第三輪回答:", response3.text)


第三輪回答: 美國隊贏過**一次**世界棒球經典賽冠軍。

他們是在**2017年**贏得冠軍的。


In [19]:
# 如果 第二輪問答時 是問 2018 年
chat_new = client.chats.create(
    model='gemini-2.5-flash',
    config=types.GenerateContentConfig(
        system_instruction="You are a helpful assistant."
    )
)
chat_new.send_message("誰贏得2013年的世界棒球經典賽冠軍?")
response4 = chat_new.send_message("那2018年呢?")
print("2018年回答:", response4.text)


2018年回答: 2018年並沒有舉辦世界棒球經典賽。

世界棒球經典賽通常是每四年舉辦一次。在2013年之後，下一屆經典賽是在**2017年**舉辦的。

**2017年世界棒球經典賽的冠軍是美國隊**，他們在決賽中擊敗了波多黎各。


In [21]:
# 換一種問法 減少幻覺現象
chat_better = client.chats.create(
    model='gemini-2.5-flash',
    config=types.GenerateContentConfig(
        system_instruction="You are a helpful assistant."
    )
)
chat_better.send_message("誰贏得2013年的世界棒球經典賽冠軍?")
response5 = chat_better.send_message("那2018年呢? 如果沒舉辦，請回答沒舉辦")
print("改善後的2018年回答:", response5.text)


改善後的2018年回答: 2018年沒有舉辦世界棒球經典賽。

世界棒球經典賽通常是四年舉辦一次。在2017年舉辦之後，下一次是在2023年（原定2021年，因疫情延期）。


In [22]:
# 出處: https://www.promptingguide.ai/zh/techniques/fewshot
prompt = f"""
請判斷情緒:

input: 這太棒了！
output: Positive

input: 這太糟糕了！
output: Negative

input: 哇，那部電影太棒了！
output: Positive

input: 多麼可怕的節目
output:
"""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt
)
print(response.text)


Negative


In [23]:
# *在一些較難描述明確指示的任務中，蠻適合用* few-shot 的方式讓模型自己學，例如文字風格、特定的輸出結構(某種schema)

# 沒給範例
prompt = f"""
晶晶體是一種流行於臺灣以中文為基底，夾雜英語不成句的單字或片語的表達方式。特指所使用的英文字多為過於簡單、沒有替換必要者，進而產生有意炫耀雙語能力卻弄巧成拙的效果。

原文: 每位員工都要參加每週電話會議，沒有例外
晶晶體:
"""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt
)
print(response.text)


這裡提供幾個版本的晶晶體：

**版本一 (最經典、最符合定義的):**
每位 **employee** 都要 **join** 每週的 **call**，沒有 **exception**。

**版本二 (替換更多詞彙):**
所有的 **staff** 都 **need to attend** 每週的 **meeting**，**no exception**。

**版本三 (更強調「必要」):**
每個 **employee** 都 **must** 參加每週的 **phone call**，**definitely no exception**。


In [24]:
# 給範例讓模型學風格，可以學得更好
prompt = f"""
晶晶體是一種流行於臺灣以中文為基底，夾雜英語不成句的單字或片語的表達方式。特指所使用的英文字多為過於簡單、沒有替換必要者，進而產生有意炫耀雙語能力卻弄巧成拙的效果。
例如:

原文: 我很忙，因為我很有事要做
晶晶體: 我是很busy，因為我很多things要do

原文: 天氣總算放晴，沒有下雨、太陽很大、有點熱、讓我想到以前還是學生時，喜歡在這樣的天氣，吃一球冰淇淋，真的會讓人很高興
晶晶體: 天氣總算放晴，沒有rain、太陽很big、有點hot、讓我想到以前還是student時，喜歡在這樣的天氣，吃一球ice cream，真的會讓人很happy

原文: 每位員工都要參加每週電話會議，沒有例外
晶晶體:
"""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt
)
print(response.text)


晶晶體: 每位**employee**都要**attend weekly**的**call**，沒有**exception**。


In [25]:
# 出處: https://promptingguide.azurewebsites.net/techniques/cot
prompt = """
我去市場買了10個蘋果。我給了鄰居2個蘋果，又給修理工2個蘋果。之後，我又去買了5個蘋果，然後吃了1個。我還剩下多少個蘋果？
"""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt
)
print(response.text)


我們來一步步計算一下：

1.  你最初有：10個蘋果
2.  給了鄰居2個：10 - 2 = 8個
3.  又給了修理工2個：8 - 2 = 6個
4.  後來又買了5個：6 + 5 = 11個
5.  最後吃了1個：11 - 1 = 10個

所以，你還剩下 **10** 個蘋果。


In [26]:
# Few-shot CoT - 給一個推理範例，也就是 Chain of Thought (CoT) 思考過程
prompt = """
Q: 我去市場買了6個香蕉，給了朋友3個香蕉，我還剩下多少個?
A:
  1. 我一開始有6個
  2. 給了朋友3個，所以剩下 6-3=3個香蕉
  3. 最後剩下3個香蕉

Q: 我去市場買了10個蘋果。我給了鄰居2個蘋果，又給修理工2個蘋果。之後，我又去買了5個蘋果，然後吃了1個。我還剩下多少個蘋果？
A:
"""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt
)
print(response.text)

1.  **最初購買的數量：** 我一開始有10個蘋果。
2.  **給了鄰居後：** 給了鄰居2個，剩下 10 - 2 = 8 個蘋果。
3.  **再給了修理工後：** 又給了修理工2個，剩下 8 - 2 = 6 個蘋果。
4.  **再次購買後：** 又去買了5個，所以現在有 6 + 5 = 11 個蘋果。
5.  **吃了一個後：** 吃了1個，所以剩下 11 - 1 = 10 個蘋果。
6.  **最後剩下10個蘋果。**


In [27]:
# Zero-shot CoT (讓模型自己想步驟)
# 標準咒語是 Let's think step by step
prompt = """
我去市場買了10個蘋果。我給了鄰居2個蘋果，又給修理工2個蘋果。之後，我又去買了5個蘋果，然後吃了1個。我還剩下多少個蘋果？
Let's think step by step
"""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt
)
print(response.text)

好的，我們一步一步來計算：

1.  **一開始買了**：10個蘋果
2.  **給了鄰居**：10 - 2 = 8個蘋果
3.  **又給了修理工**：8 - 2 = 6個蘋果
4.  **後來又買了**：6 + 5 = 11個蘋果
5.  **吃掉了**：11 - 1 = 10個蘋果

你還剩下 **10** 個蘋果。


In [28]:
prompt = "請隨機產生三個 user 資料，請用 JSON 格式回傳"

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt
)
print(response.text)

好的，這是隨機產生三筆 user 資料的 JSON 格式：

```json
[
  {
    "id": "usr-001",
    "username": "john_doe",
    "email": "john.doe@example.com",
    "age": 30,
    "isActive": true,
    ""registeredDate": "2023-01-15T10:30:00Z"
  },
  {
    "id": "usr-002",
    "username": "jane_smith",
    "email": "jane.smith@example.com",
    "age": 24,
    "isActive": true,
    "registeredDate": "2022-07-20T14:00:00Z"
  },
  {
    "id": "usr-003",
    "username": "peter_chen",
    "email": "peter.chen@example.com",
    "age": 45,
    "isActive": false,
    "registeredDate": "2021-03-01T08:45:00Z"
  }
]
```


In [29]:
# Gemini 的 Structured Output 功能
# 文件: https://ai.google.dev/gemini-api/docs/json-mode

from google.genai import types

# 定義 JSON Schema
response_schema = {
    "type": "OBJECT",
    "properties": {
        "name": {
            "type": "STRING",
            "description": "請用台灣常見姓名"
        },
        "age": {
            "type": "INTEGER",
            "description": "年紀"
        },
        "bio": {
            "type": "STRING",
            "description": "請用台灣繁體中文"
        },
        "avatar_url": {
            "type": "STRING",
            "description": "個人圖像，請用真實可以連結的圖片"
        },
        "isSubscriber": {
            "type": "BOOLEAN",
            "description": "是否訂閱"
        }
    },
    "required": [
        "name",
        "age",
        "bio",
        "avatar_url",
        "isSubscriber"
    ]
}

# 使用 response schema
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt,
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=response_schema
    )
)
print(response.text)


{
  "name": "陳淑芬",
  "age": 32,
  "bio": "來自台北的軟體工程師，熱愛爬山和攝影，喜歡探索台灣的自然美景。",
  "avatar_url": "https://picsum.photos/id/1012/200/300",
  "isSubscriber": true
}


In [30]:
# 使用 Pydantic 定義 schema 更方便
from typing import List
from pydantic import Field, BaseModel, ConfigDict

class User(BaseModel):
    name: str = Field(description="請用台灣常見姓名")
    age: int = Field(description="年紀")
    bio: str = Field(description="請用台灣繁體中文")
    avatar_url: str = Field(description="個人圖像，請用真實可以連結的圖片")
    isSubscriber: bool = Field(description="是否訂閱")

class Users(BaseModel):
    users: list[User]

# 將 Pydantic model 轉換為 Gemini 可用的 schema
def pydantic_to_gemini_schema(pydantic_model):
    schema = pydantic_model.model_json_schema()
    return schema

users_schema = pydantic_to_gemini_schema(Users)
print("Generated Schema:", users_schema)


Generated Schema: {'$defs': {'User': {'properties': {'name': {'description': '請用台灣常見姓名', 'title': 'Name', 'type': 'string'}, 'age': {'description': '年紀', 'title': 'Age', 'type': 'integer'}, 'bio': {'description': '請用台灣繁體中文', 'title': 'Bio', 'type': 'string'}, 'avatar_url': {'description': '個人圖像，請用真實可以連結的圖片', 'title': 'Avatar Url', 'type': 'string'}, 'isSubscriber': {'description': '是否訂閱', 'title': 'Issubscriber', 'type': 'boolean'}}, 'required': ['name', 'age', 'bio', 'avatar_url', 'isSubscriber'], 'title': 'User', 'type': 'object'}}, 'properties': {'users': {'items': {'$ref': '#/$defs/User'}, 'title': 'Users', 'type': 'array'}}, 'required': ['users'], 'title': 'Users', 'type': 'object'}


In [31]:
# 使用 Pydantic schema 創建請求
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents="請隨機產生多個 user 資料",
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=users_schema
    )
)
print(response.text)

# 解析回應到 Pydantic 物件
import json
response_data = json.loads(response.text)
parsed_users = Users(**response_data)
print("\n解析後的第一個用戶:", parsed_users.users[0])


{"users": [{"name": "陳美玲", "age": 28, "bio": "我是美玲，喜歡閱讀和手作烘焙。假日經常在咖啡廳度過，享受一杯好咖啡的時光。", "avatar_url": "https://i.imgur.com/8Q9K2mH.jpeg", "isSubscriber": true}, {"name": "林志明", "age": 35, "bio": "我叫志明，是個程式設計師，熱愛學習新技術。下班後喜歡打籃球或爬山，保持身心健康。", "avatar_url": "https://i.imgur.com/G4Y2B7K.jpeg", "isSubscriber": false}, {"name": "王雅婷", "age": 22, "bio": "大家好，我是雅婷。目前是一名大學生，主修設計。平常喜歡畫畫和聽音樂，夢想是成為一名插畫家。", "avatar_url": "https://i.imgur.com/5J3K7pM.jpeg", "isSubscriber": true}]}

解析後的第一個用戶: name='陳美玲' age=28 bio='我是美玲，喜歡閱讀和手作烘焙。假日經常在咖啡廳度過，享受一杯好咖啡的時光。' avatar_url='https://i.imgur.com/8Q9K2mH.jpeg' isSubscriber=True


In [45]:
# Step 1: 從用戶問題中，用 prompt1 來提取出 外部工具的參數
from typing import List
from pydantic import Field, BaseModel

class QueryResult(BaseModel):
    date: str = Field(description="Date in yyyymmdd format. Leave empty if not parsable")
    stock_code: str = Field(description="Taiwan stock code as a 4-digit number. Leave empty if not parsable")

query = "請問2025年6月18號的「台積電」，股價表現如何?"

# 使用參數提取
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=query,
    config=types.GenerateContentConfig(
        system_instruction="Extract from user queries",
        response_mime_type="application/json",
        response_schema=pydantic_to_gemini_schema(QueryResult)
    )
)

extracted_data = json.loads(response.text)
parsed_result = QueryResult(**extracted_data)
print("提取的參數:", parsed_result)


提取的參數: date='20250618' stock_code='2330'


In [46]:
# Step 2: 執行工具，拿到結果
# API 參考自 https://medium.com/%E5%B7%A5%E7%A8%8B%E9%9A%A8%E5%AF%AB%E7%AD%86%E8%A8%98/5%E7%A8%AE%E6%8A%93%E5%8F%96%E5%8F%B0%E8%82%A1%E6%AD%B7%E5%8F%B2%E8%82%A1%E5%83%B9%E7%9A%84%E6%96%B9%E6%B3%95-766bf2ed9d6

import requests
import json

date = parsed_result.date
stock_code = parsed_result.stock_code
url = 'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=%s&stockNo=%s' % (date, stock_code)

html = requests.get(url)
context = json.loads(html.text)
print("取得的股價資料:", context)


取得的股價資料: {'stat': 'OK', 'date': '20250618', 'title': '114年06月 2330 台積電           各日成交資訊', 'fields': ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數'], 'data': [['114/06/02', '40,608,468', '38,643,155,297', '958.00', '961.00', '946.00', '946.00', '-21.00', '125,245'], ['114/06/03', '27,482,916', '26,302,041,072', '960.00', '965.00', '950.00', '950.00', '+4.00', '43,550'], ['114/06/04', '43,196,396', '42,419,360,833', '974.00', '990.00', '970.00', '990.00', '+40.00', '73,539'], ['114/06/05', '27,047,750', '26,936,792,104', '1,000.00', '1,000.00', '991.00', '998.00', '+8.00', '51,309'], ['114/06/06', '18,797,346', '18,688,985,614', '997.00', '997.00', '991.00', '995.00', '-3.00', '27,283'], ['114/06/09', '23,952,041', '24,100,014,375', '1,005.00', '1,010.00', '1,000.00', '1,005.00', '+10.00', '56,903'], ['114/06/10', '55,353,908', '57,406,744,645', '1,025.00', '1,050.00', '1,020.00', '1,045.00', '+40.00', '138,656'], ['114/06/11', '46,137,188', '48,960,267,029', '1,065.00

In [47]:
# Step 3: 用 (prompt2 + 結果) 轉成自然語言回給用戶
prompt = f"""
Based on the provided context, please answer the following question in Traditional Chinese (Taiwan):

Question: <question>{query}</question>

Context: <context>{context}</context>

Instructions:
1. Carefully verify that your answer is supported by the given context.
2. If the question cannot be answered from the provided context, respond with: "抱歉，在提供的資料中找不到相關資訊，無法回答您的問題。"
3. Do not include information that is not present in the context.
4. Ensure your response is written in Traditional Chinese as used in Taiwan.
"""

response = model.generate_content(prompt)
print("最終回答:", response.text)


最終回答: 根據提供的資料，2025年6月18日（114年06月18日）台積電的股價表現如下：

*   **開盤價**：1,040.00元
*   **最高價**：1,055.00元
*   **最低價**：1,030.00元
*   **收盤價**：1,055.00元
*   **漲跌價差**：上漲10.00元
*   **成交股數**：39,423,315股
*   **成交金額**：41,289,195,000元
*   **成交筆數**：45,603筆


## 案例: 長文產生器，拆解多個子主題，分開搜尋回答後，最後整合在一起


根據用戶輸入的主題生成一篇全面綜述文章，流程是

1. 根據用戶輸入的主題，拆解成多個子主題
2. 針對每個子主題，進行網路搜尋出參考資料，然後生成獨立的子文章
3. 將所有子文章整合並潤飾成一篇連貫的長文。

其中步驟 (2) 是平行執行

和上一個摘要的 Parallelization 的差異在於，這裏的第一步 Orchestrator 會用 AI 來拆解出不固定的子任務。

<img src="https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2F8985fc683fae4780fb34eab1365ab78c7e51bc8e-2401x1000.png&w=3840&q=75">


In [3]:
!pip install tavily-python

In [19]:
import asyncio
from datetime import datetime
from tavily import TavilyClient
from google.genai import types

tavily_api_key = userdata.get('tavily_api_key')
tavily_client = TavilyClient(api_key=tavily_api_key)

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


In [28]:
# Step 1: 定義工具函式
def web_search(keyword: str) -> str:
    response = tavily_client.search(query=keyword)
    return str(response["results"])

# Step 2: 定義函數描述給 Gemini
web_search_tool = types.Tool(
    function_declarations=[
        types.FunctionDeclaration(
            name="web_search",
            description="搜尋最新的資訊",
            parameters=types.Schema(
                type=types.Type.OBJECT,
                properties={
                    "keyword": types.Schema(
                        type=types.Type.STRING,
                        description="搜尋關鍵字"
                    )
                },
                required=["keyword"]
            )
        )
    ]
)

# Step 3: 發送請求 (使用新的 client.models.generate_content)
today = datetime.now().strftime("%Y/%m/%d")

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=f"今天 {today} 台北的天氣如何？",
    config=types.GenerateContentConfig(
        tools=[web_search_tool]
    )
)

print("模型回應:")
print(response.text)

from pprint import pp
print("\nResponse: ")
pp(response)

print("\nFunction Calls:")

for candidate in response.candidates:
    for part in candidate.content.parts:
        if hasattr(part, 'function_call') and part.function_call:
            print(f"Function: {part.function_call.name}")
            print(f"Arguments: {dict(part.function_call.args)}")

            # Step 4: 執行函數調用
            if part.function_call.name == "web_search":
                keyword = part.function_call.args["keyword"]
                result = web_search(keyword)
                print(f"Function Result: {result}")

                # Step 5: 將結果回傳給模型
                final_response = client.models.generate_content(
                    model='gemini-2.5-flash',
                    contents=[
                        f"今天 {today} 台北的天氣如何？",
                        types.Content(
                            parts=[types.Part(function_call=part.function_call)]
                        ),
                        types.Content(
                            parts=[types.Part(
                                function_response=types.FunctionResponse(
                                    name="web_search",
                                    response={"result": result}
                                )
                            )]
                        )
                    ]
                )
                print(f"\n最終回應: {final_response.text}")

模型回應:
None

Response: 
GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id=None, args={'keyword': '台北天氣 2025/06/19'}, name='web_search'), function_response=None, text=None)], role='model'), citation_metadata=None, finish_message=None, token_count=None, finish_reason=<FinishReason.STOP: 'STOP'>, url_context_metadata=None, avg_logprobs=None, grounding_metadata=None, index=0, logprobs_result=None, safety_ratings=None)], create_time=None, response_id=None, model_version='models/gemini-2.5-flash-preview-05-20', prompt_feedback=None, usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=27, candidates_tokens_details=None, prompt_token_count=57, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT

In [35]:
# 可用的函數字典
available_functions = {
    "web_search": web_search
}

# 使用 google.genai 時，可以直接傳入 Python 函數作為工具
tools = [web_search]

# 建立模型配置 (google.genai 使用不同的方式)
model_name = "gemini-2.5-flash"  # 或其他支援的模型名稱

# 包成一個完整的 function calling 輔助函數
def run_gemini_with_functions(client, model_name, user_message, tools):
    """執行帶有函數調用的 Gemini 請求"""

    print(f"💬 用戶問題: {user_message}")

    # 第一次調用 - 使用 google.genai 的方式
    response = client.models.generate_content(
        model=model_name,
        contents=user_message,
        config=types.GenerateContentConfig(
            tools=tools
        )
    )

    # 檢查是否有函數調用
    if hasattr(response, 'candidates') and response.candidates:
        candidate = response.candidates[0]
        if hasattr(candidate, 'content') and hasattr(candidate.content, 'parts'):
            # 檢查是否有函數調用
            function_calls = [part for part in candidate.content.parts if hasattr(part, 'function_call')]

            if function_calls:
                print("🔧 檢測到函數調用")
                # google.genai 支持自動函數調用
                # 如果工具是 Python 函數，SDK 會自動處理調用
                return response.text
            else:
                # 沒有函數調用，直接返回
                return response.text

    return response.text if hasattr(response, 'text') else str(response)

# 測試這個輔助函數
result = run_gemini_with_functions(
    client,
    model_name,
    "今天台北、新竹、高雄的天氣如何？",
    tools
)
print("------")
print("最終結果:", result)


💬 用戶問題: 今天台北、新竹、高雄的天氣如何？
🔧 檢測到函數調用
------
最終結果: **台北：**
今天氣溫約為 27-36°C (81-97°F)，感覺炎熱。部分時間多雲，午後可能有短暫雷陣雨，降雨機率約為 4% 至 84% 不等。空氣品質不佳。

**新竹：**
今天多雲到晴，氣溫約 25-32°C (77-90°F)，體感舒適到悶熱。有機會出現降雨，降雨機率約為 20% 至 51%，有時可能有大雨。

**高雄：**
今天晴朗，氣溫約 26-32°C (79-90°F)，體感炎熱。午後有局部短暫雷陣雨的機會，降雨機率約為 7% 至 70%。空氣品質大致良好。


In [36]:
# 圖像處理範例
# 首先我們需要上傳一個圖像文件到 Gemini

# 方法1: 從 URL 載入圖像 (需要 Pillow)
%pip install Pillow

import PIL.Image
import requests
from io import BytesIO

# 下載一個範例圖像
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
response = requests.get(image_url)
image = PIL.Image.open(BytesIO(response.content))

# 使用 Gemini 分析圖像
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[
        "請詳細描述這張圖片的內容，包括景觀、顏色、氛圍等。請用繁體中文回答。",
        image
    ]
)

print("圖像分析結果:")
print(response.text)


圖像分析結果:
這張圖片呈現了一片廣闊而寧靜的自然景觀，充滿了生機與活力。

**景觀 (Landscape):**
畫面中央是一條蜿蜒向前延伸的木棧道，它筆直地引導著觀者的視線深入畫面，直到遠方消失在翠綠的田野之中。棧道兩旁是茂盛的草地，綠草如茵，高過膝蓋，隨風輕擺，營造出柔軟而豐厚的質感。近景處的草葉細節清晰可見，有些草穗在陽光下泛著金光。遠方地平線處，可見一片低矮的樹林或灌木叢，樹木蔥鬱，為景觀增添了層次感和深度。整片視野開闊，沒有明顯的人工建築，展現了純粹的自然之美。

**顏色 (Colors):**
圖片的色彩豐富而飽和，主要以藍色和綠色為主調：
*   **天空：** 佔據了畫面約三分之一的比例，呈現出清澈明亮的蔚藍色，從靠近地平線的淺藍逐漸過渡到天頂的深藍，富有層次。天空中有著輕盈的白色捲雲和絮狀雲朵，如絲帶般飄散，為藍天增添了動態與美感。
*   **草地：** 以鮮亮的翠綠色為主，尤其在近景處，綠草的細節和光澤表現得淋漓盡致。部分被陽光照耀的區域呈現出帶有金黃或淺綠色的光澤，暗示了陽光的角度和時間。遠處的草地顏色則較為均勻和深沉。
*   **木棧道：** 本身是淺灰棕色或米白色，木板的紋理清晰可見，與周圍的鮮綠形成鮮明對比，但也自然地融入了景觀之中。
*   **遠方樹木：** 主要為深綠色，但在右側遠處的灌木叢則帶有較為溫暖的黃褐色調，可能是秋季的色彩，或特定植物的顏色。

**氛圍 (Atmosphere):**
整張圖片散發著一種寧靜、平和而開闊的氛圍。陽光似乎是溫和而充足的，均勻地灑落在草地和棧道上，為畫面帶來暖意和生機。木棧道延伸至遠方，給人一種探索、漫步的邀請，彷彿能引領觀者走向未知的田園深處，感受大自然的呼吸。這是一幅充滿生命力且令人心曠神怡的田園風光，給人心靈放鬆、回歸自然的舒適感，讓人想在此駐足，享受片刻的寧靜與美好。
